In [1]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import pandas as pd
import pyprojroot
import seaborn as sns

In [2]:
def convert_seg_error_rate_pct(df):
    df.avg_segment_error_rate = df.avg_segment_error_rate * 100
    return df

In [3]:
RESULTS_ROOT = pyprojroot.here() / 'results'

FIG_ROOT = pyprojroot.here() / 'doc' / 'article' / 'figures' / 'mainfig_tweetynet_v_svm'
FIG_ROOT.mkdir(exist_ok=True)

#### Munge data

In [4]:
segmentation_map = {
    'ground_truth': 'segmented audio, manually cleaned',
    'resegment': 'segmented audio, not cleaned',
    'semi-automated-cleaning': 'segmented audio, semi-automated cleaning',
    'not-cleaned': 'segmented audio, not cleaned',
    'manually-cleaned': 'segmented audio, manually cleaned'
}

In [5]:
hvc_dfs = []


csv_filename = 'segment_error_across_birds.hvc.csv'

for species in ('Bengalese_Finches', 'Canaries'):
    species_csv = RESULTS_ROOT / f'{species}/hvc/{csv_filename}'
    df = pd.read_csv(species_csv)

    df['Model'] = 'SVM'
    df['Input to model'] = df['segmentation'].map(segmentation_map)
    df['Species'] = species
    hvc_dfs.append(df)

hvc_df = pd.concat(hvc_dfs)

In [6]:
curve_df = []

for species in ('Bengalese_Finches', 'Canaries'):
    LEARNCURVE_RESULTS_ROOT = pyprojroot.here() / 'results' / species / 'learncurve'
    error_csv_path = LEARNCURVE_RESULTS_ROOT.joinpath('error_across_birds_with_cleanup.csv')
    df = pd.read_csv(error_csv_path)

    df = df[df.animal_id.isin(hvc_df.animal_id.unique())]
    df['Model'] = 'TweetyNet'
    df['Input to model'] = 'spectrogram'
    df['Species'] = species
    curve_df.append(df)

del df
curve_df = pd.concat(curve_df)

In [7]:
CLEANUP = 'min_segment_dur_majority_vote'

curve_df = curve_df[
    curve_df.cleanup == CLEANUP
]

In [8]:
all_df = pd.concat([hvc_df, curve_df])

all_df = convert_seg_error_rate_pct(all_df)

In [9]:
all_df.head()  # sanity check

Unnamed: 0 animal_id  train_set_dur  replicate_num  \
0         0.0  bl26lb16             75              1   
1         1.0  bl26lb16             75              1   
2         2.0  bl26lb16             75              1   
3         3.0  bl26lb16             75              2   
4         4.0  bl26lb16             75              2   

              segmentation  avg_segment_error_rate Model  \
0              not-cleaned                6.406168   SVM   
1  semi-automated-cleaning                0.924909   SVM   
2         manually-cleaned                0.369642   SVM   
3              not-cleaned                6.349347   SVM   
4  semi-automated-cleaning                0.831458   SVM   

                             Input to model            Species cleanup  ...  \
0              segmented audio, not cleaned  Bengalese_Finches     NaN  ...   
1  segmented audio, semi-automated cleaning  Bengalese_Finches     NaN  ...   
2         segmented audio, manually cleaned  Bengalese_Finches     NaN  ...   
3              segmented audio, not cleaned  Bengalese_Finches     NaN  ...   
4  segmented audio, semi-automated cleaning  Bengalese_Finches     NaN  ...   

   avg_pct_boundary_err  model_name  results_dir toml_path  \
0                   NaN         NaN          NaN       NaN   
1                   NaN         NaN          NaN       NaN   
2                   NaN         NaN          NaN       NaN   
3                   NaN         NaN          NaN       NaN   
4                   NaN         NaN          NaN       NaN   

  training_replicate_csv_path checkpoint_path spect_scalar_path  \
0                         NaN             NaN               NaN   
1                         NaN             NaN               NaN   
2                         NaN             NaN               NaN   
3                         NaN             NaN               NaN   
4                         NaN             NaN               NaN   

  min_segment_dur avg_error  train_set_dur_ind  
0             NaN       NaN                NaN  
1             NaN       NaN                NaN  
2             NaN       NaN                NaN  
3             NaN       NaN                NaN  
4             NaN       NaN                NaN  

[5 rows x 22 columns]

In [10]:
gb = all_df.groupby(by=['Species', 'Model', 'Input to model', 'animal_id', 'train_set_dur'])
df_agg = gb.agg(
    mean_seg_err = pd.NamedAgg('avg_segment_error_rate', 'mean'),
    median_seg_err = pd.NamedAgg('avg_segment_error_rate', 'median'),
    std_seg_err = pd.NamedAgg('avg_segment_error_rate', 'std')
)

data = df_agg.reset_index()  # ``data`` DataFrame for use with ``seaborn``

In [11]:
PALETTE = sns.color_palette('colorblind')  # note: defaults to 10 colors

MODEL_HUE_MAP = {
    'TweetyNet': PALETTE[0],
    'SVM': PALETTE[1],
}

DASHES = {
    'segmented audio, manually cleaned': (4, 1.5),
    'segmented audio, not cleaned': (1, 1),
    'segmented audio, semi-automated cleaning': (3, 1.25, 1.5, 1.25),
    'spectrogram': '',
}

#### actually plot figure

In [12]:
# specify the custom font to use
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'arial'

sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 3})

figsize = (8, 4)
dpi = 150

fig = plt.figure(constrained_layout=False, figsize=figsize, dpi=dpi)

gs = fig.add_gridspec(
    nrows=3, 
    ncols=2, 
)

ax_arr = []
for rows in (
    (0, 1), (1, 3),
):
    start_row, stop_row = rows
    for col in range(2):
        ax_arr.append(fig.add_subplot(gs[start_row:stop_row, col]))

ax_arr = np.array(ax_arr).reshape(2, 2)

MIN_TRAIN_SET_DUR_CANARY = 240

for col, species in enumerate(('Bengalese_Finches', 'Canaries')):
    # ---- get data
    data_species = data[data.Species == species]
    
    if species == 'Canaries':
        data_species = data_species[data_species.train_set_dur >= MIN_TRAIN_SET_DUR_CANARY]

    train_set_durs = sorted(data_species['train_set_dur'].unique())
    dur_int_map = dict(zip(train_set_durs, range(len(train_set_durs))))
    data_species['train_set_dur_ind'] = data_species['train_set_dur'].map(dur_int_map)

    TRAIN_DUR_IND_MAP = {
    k:v for k, v in zip(
        sorted(data_species['train_set_dur'].unique()), 
        sorted(data_species['train_set_dur_ind'].unique())
    )
    }

    svm_mean_seg_err_max_dur = data_species[
        (data_species.Model=='SVM') & (data_species['Input to model'] == 'segmented audio, manually cleaned') & (data_species.train_set_dur ==  data_species.train_set_dur.max())
    ].mean_seg_err.mean()

    tweetynet_mean_seg_err_max_dur = data_species[
        (data_species.Model=='TweetyNet') & (data_species['Input to model'] == 'spectrogram') & (data_species.train_set_dur ==  data_species.train_set_dur.max())
    ].mean_seg_err.mean()
    
    # ---- set up to plot
    col_ax_arr = ax_arr[:, col]

    col_ax_arr[0].get_shared_x_axes().join(col_ax_arr[1])

    # we plot the same data on both axes, and then change the ylims below
    for row, ax in enumerate(col_ax_arr):
        if col == 1 and row == 1:
            # let seaborn generate legend, then we get handles + labels and place under whole figure
            legend = True
        else:
            legend = False

        g = sns.lineplot(
            data=data_species,
            x='train_set_dur_ind',
            y='mean_seg_err',
            hue='Model',
            palette=MODEL_HUE_MAP,
            style='Input to model',
            dashes=DASHES,
            ax=ax,
            legend=legend,
        )

        ax.set_xlabel('')
        ax.set_ylabel('')

        
        if col == 1 and row == 1:
            handles, labels = ax.get_legend_handles_labels()
            g.legend_.remove()

    col_ax_arr[0].set_ylim([12, 105.])
    col_ax_arr[1].set_ylim([0., 11])
    
    col_ax_arr[0].set_xticks(list(TRAIN_DUR_IND_MAP.values()))
    col_ax_arr[0].set_xticklabels([])
    col_ax_arr[1].set_xticks(list(TRAIN_DUR_IND_MAP.values()))
    col_ax_arr[1].set_xticklabels(train_set_durs, rotation=45)

    col_ax_arr[0].set_title(species.replace('_', ' '))

# --- despine and fix ticks
for ax_ind, spines, tick_params in zip(
    (0, 1, 2, 3),
    (
        ('top', 'bottom', 'right'),
        ('top', 'bottom', 'right'),
        ('top', 'right',),
        ('top', 'right')
    ),
    (
        dict(axis='x', bottom=False),
        dict(axis='x', bottom=False),
        None,
        None,
    )
):
    for spine in spines:
        ax_arr.flatten()[ax_ind].spines[spine].set_visible(False)
    if tick_params is not None:
        ax_arr.flatten()[ax_ind].tick_params(**tick_params)

# # add a big axis, hide frame
big_ax = fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
big_ax.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
big_ax.grid(False)

big_ax.set_xlabel('Training set duration (s)', labelpad=20)
big_ax.set_ylabel('Syllable error rate (%)', labelpad=15)

big_ax.legend(handles, labels, loc='upper left', bbox_to_anchor=[0.0, -0.25])
    
FIG_STEM = 'svm-v-tweetynet-results'
    
for ext in ('png', 'svg'):
    fig.savefig(FIG_ROOT / f'{FIG_STEM}.{ext}', bbox_inches='tight');

<ipython-input-12-4c0699352bcb>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_species['train_set_dur_ind'] = data_species['train_set_dur'].map(dur_int_map)
